In [1]:
!git clone https://github.com/jherberg462/SIIM-ISIC.git --depth 1

Cloning into 'SIIM-ISIC'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [2]:
import os
os.chdir('SIIM-ISIC')

In [3]:
params = {
    'batch_size' : 128,
    'img_size' : 256, #length and width will be equal
    'epochs': 400
}

In [4]:
from create_params import update_env_variables
update_env_variables(params)

In [5]:
import numpy as np 
import pandas as pd 


import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow import keras
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

try:
    from kaggle_datasets import KaggleDatasets
    dataset_gcs = KaggleDatasets().get_gcs_path('siim-isic-melanoma-classification')
    print('got GCS path via KaggleDatasets .get_gcs_path method')
except ModuleNotFoundError:
    #hardcode path while testing locally
    dataset_gcs = 'gs://kds-599205fd0d8963558ce1308147ba090f776d31b1662a67f2ddccfa38'

#
from input_pipeline import (decode_image_label, decode_image, 
                            normalize_image_label, random_flip, 
                            get_train_ds, get_test_ds, get_ds_size)
from params import params

got GCS path via KaggleDatasets .get_gcs_path method


ImportError: cannot import name 'params_scratch' from 'params' (/kaggle/working/SIIM-ISIC/params.py)

In [ ]:
tf.__version__


In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# sub = pd.read_csv(dataset_gcs + '/sample_submission.csv')
# sub.head(1)

In [ ]:
train_df = pd.read_csv(dataset_gcs + '/train.csv')
train_df.groupby('target').count() 

In [ ]:
# def decode_image_label(tfrec):
#     '''
#     function to decode an image and target label from tfrecord
    
#     args:
#         tfrec: tfrecord, single record of training/validation data
    
#     returns:
#         decoded_image: tensor, converted image from tfrecord
#         label: tensor, integer, either 1 or 0
    
#     '''
    
#     features_dictionary = {
#         'image': tf.io.FixedLenFeature([], tf.string),
#         "target": tf.io.FixedLenFeature([], tf.int64)
#         }
#     features = tf.io.parse_single_example(tfrec, features_dictionary)
#     decoded_image = tf.io.decode_jpeg(features['image'], 3)
#     decoded_image = tf.image.resize(decoded_image, params['img_size'])
#     label = features['target']
    
#     return decoded_image, label
    


In [ ]:
# def decode_image(tfrec):
#     '''
#     function to decode an image from tfrecord
    
#     args:
#         tfrec: tfrecord, single record of training/validation data
    
#     returns:
#         decoded_image: tensor, converted image from tfrecord
#         img_name: tensor, string, Id of the decoded image
    
#     '''
    
#     features_dictionary = {
#         'image': tf.io.FixedLenFeature([], tf.string),
#         "image_name": tf.io.FixedLenFeature([], tf.string)
#         }
#     features = tf.io.parse_single_example(tfrec, features_dictionary)
#     decoded_image = tf.io.decode_jpeg(features['image'], 3)
#     decoded_image = tf.image.resize(decoded_image, params['img_size'])
#     img_name = features['image_name']
    
#     return decoded_image, img_name

In [ ]:
# def normalize_image_label(decoded_image, label):
#     '''
#     function to convert an image tensor values from 0 to 255 
#     -> -1 to 1
#     to be used when dealing with tfrecords containing labels
    
#     args:
#         decoded_image: tensor that is an image with values from 0 to 255
#         label: tensor, target label
    
#     returns: 
#         image_tensor: tensor that is an image with values from -1 to 1
#         label, same as input
    
#     '''
    
#     #add dim at the zero axis Shape will be from (x, y, z) -> (None, x, y, z)
#     image_tensor = tf.expand_dims(decoded_image, 0)
#     #undo the above line -- this is needed due to TF not allowing a filtered tensor py_function
#     image_tensor = tf.gather(image_tensor, 0)

#     #convert tensor values to between -1 and 1 (0 to 255 -> -1 to 1)
#     image_tensor = (tf.cast(image_tensor, tf.float32) - 127.5) / 127.5

#     return image_tensor, label

In [ ]:
# ***not needed? ***
# def normalize_image(decoded_image):
#     '''
#     function to convert an image tensor values from 0 to 255 
#     -> -1 to 1
    
#     args:
#         decoded_image: tensor that is an image with values from 0 to 255
    
#     returns: 
#         image_tensor: tensor that is an image with values from -1 to 1
    
#     '''
    
#     #add dim at the zero axis Shape will be from (x, y, z) -> (None, x, y, z)
#     image_tensor = tf.expand_dims(decoded_image, 0)
#     #undo the above line -- this is needed due to TF not allowing a filtered tensor py_function
#     image_tensor = tf.gather(image_tensor, 0)

#     #convert tensor values to between -1 and 1 (0 to 255 -> -1 to 1)
#     image_tensor = (tf.cast(image_tensor, tf.float32) - 127.5) / 127.5

#     return image_tensor

In [ ]:
# def random_flip(image, label):
#     '''
#     function to randomly flip images on the x and/or y axis
    
#     args:
#         image: tensor, an image
#         label: tensor, target label
    
#     returns: 
#         image: tensor, same as input, but possibly flipped on x and/or y axis
#         label, tensor, same as input
#     '''
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
#     return image, label  

In [ ]:
# def get_train_ds(tfrecords, batch_size):
#     '''
#     function to create a ds pipeline from tfrecord files
    
#     args:
#         tfrecords: list, tfrecord file paths
#         batch_size: int, batch size for number of records to pass into
#             model at a time
#     returns:
#         ds: tensorflow input pipeline with images and labels
#     '''
#     ds = (tf.data.TFRecordDataset(filenames=[tfrecords],
#                                  num_parallel_reads=tf.data.experimental.AUTOTUNE).
#           cache(). #need to remove cache while not usnig TPUs
#           map(decode_image_label, num_parallel_calls=tf.data.experimental.AUTOTUNE).
#           map(normalize_image_label, num_parallel_calls=tf.data.experimental.AUTOTUNE).
#           map(random_flip, num_parallel_calls=tf.data.experimental.AUTOTUNE).
#           repeat().
#           shuffle(512).
#           batch(batch_size,
#                drop_remainder=True).
#           prefetch(tf.data.experimental.AUTOTUNE)
#          )
    

    
#     return ds
    

In [ ]:
# def get_test_ds(tfrecords, batch_size):
#     '''
#     function to create a dataset for test data
#     args:
#         tfrecords: list, tfrecord file paths
#         batch_size: int, batch size for number of records to pass into
#             model at a time
#     returns:
#         ds: tensorflow input pipeline with images and labels
    
#     '''

#     ds = (tf.data.TFRecordDataset(filenames=[tfrecords],
#                                  num_parallel_reads=tf.data.experimental.AUTOTUNE).
# #           cache(). #there is no reason to cache this ds -- it is only being read 1x
#           map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).
#           map(normalize_image_label, num_parallel_calls=tf.data.experimental.AUTOTUNE).
# #           map(random_flip).
#           batch(batch_size).
# #                 drop_remainder=True).
#           prefetch(tf.data.experimental.AUTOTUNE)
#          )
    
#     return ds
#     ###come back to this

# model architecture

In [ ]:
def set_of_layers(model_, filters_, kernal, strides_, dropout=0):
    '''
    function to add the following layers to a model:
    Conv2D, MaxPooling2D, BatchNormalization, LeadyReLU, Dropout

    args:
      model_ : tf.keras.Sequential model
      filters_: int, number of filters in Conv2D layer
      kernal: int, kernal size in Conv2D layer
      strides_: int, stride size in MaxPooling2D layer
      dropout: float, dropout percentage in Dropout layer, default is 0.0
        must be less than 1.0

    returns:
      model_: tf.keras.Sequential model that is the same as the model_ input plus above 
        layers added
    '''
    model_.add(layers.Conv2D(filters_, (kernal, kernal), padding='same'))
    model_.add(layers.MaxPooling2D(strides_, strides_))
    model_.add(layers.BatchNormalization())
    model_.add(layers.ReLU()) #also try ReLU // LeakyReLU
    model_.add(layers.Dense(16)) #try activation fn here
    model_.add(layers.Dropout(0.5)) #hold off on this for now //hardcode at 0.1

    return model_

In [ ]:
def deconv_set_of_layers(model_, filters_, kernal_, stride):
    '''
    function to add the following layers to a model:
    Conv2DTranspose, BatchNormalization, LeadyReLU, Dense

    args:
      model_ : tf.keras.Sequential model
      filters_: int, number of filters in Conv2DTranspose layer
      kernal_: int, kernal size in Conv2DTranspose layer
      strides_: int, stride size in Conv2DTranspose layer


    returns:
      model_: tf.keras.Sequential model that is the same as the model_ input plus above 
        layers added
    '''
    model_.add(layers.Conv2DTranspose(filters_,
                                     kernal_,
                                     (stride, stride),
                                     padding='same'))
    model_.add(layers.BatchNormalization())
    model_.add(layers.Dense(16, activation='tanh')) #keep this hardcoded for now
    model_.add(layers.Dropout(0.5))
    return model_


In [ ]:
def create_model(input_shape=[*params['img_size'], 3], bias_output=None):
    '''
    function to create a model that will be trained on train DS
    
    args:
        input_shape: array, default: [1024, 1024, 3], shape
            of input tensor that will be fed into model
    
    returns:
        model: tf.sequential() model
    '''
    model = tf.keras.Sequential()
    
    
    model.add(layers.Conv2D(32, (5, 5), padding='same',
                                     input_shape=input_shape)) 
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(16))
    model.add(layers.Dropout(.5))
    
    set_of_layers(model, 128, 5, 2)
    set_of_layers(model, 256, 5, 2)
    set_of_layers(model, 512, 5, 2)
    set_of_layers(model, 1024, 5, 2)
    deconv_set_of_layers(model, 256, 4, 2)
    deconv_set_of_layers(model, 128, 4, 2)
    deconv_set_of_layers(model, 64, 4, 2)
    
    set_of_layers(model, 64, 5, 2)
    set_of_layers(model, 128, 5, 2)
    set_of_layers(model, 256, 5, 2)
    


    
    model.add(layers.Flatten())
    model.add(layers.Dense(128))
#     model.add(layers.Dense(64))
    
    if bias_output is not None:
        bias_output = tf.keras.initializers.Constant(bias_output)
    model.add(layers.Dense(1, activation='sigmoid', bias_initializer=bias_output))
    

    metrics = [
          keras.metrics.TruePositives(name='tp'),
#           keras.metrics.FalsePositives(name='fp'),
#           keras.metrics.TrueNegatives(name='tn'),
          keras.metrics.FalseNegatives(name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
#           keras.metrics.Precision(name='precision'),
#           keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
    ]
    schedule = None
    
    model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.01),
    metrics=metrics
)
    

    
    return model



In [ ]:
def get_ds_size(files):
    '''
    function to get size of tfrecord Dataset, based on file name
    
    the file name has the number of records in the file, for example:
    train09-2071.tfrec has 2017 records
    
    args:
        files: list of str file names, each item should be the path to a tfrecord file
    
    returns:
        size: int, size of dataset
    '''
    size = 0
    for file in files:
        file_size = int(file.split('.tfrec')[0].split('tfrecords/')[1].split('-')[1])
        size += file_size
    return size

In [ ]:
#get test file paths
test_files = tf.io.gfile.glob(dataset_gcs + '/tfrecords/test*.tfrec')

#get train and validation file paths
train_files, valid_files = train_test_split(tf.io.gfile.glob(dataset_gcs + '/tfrecords/train*.tfrec'),
                              test_size=.1, random_state=1)

#create datasets
train_ds = get_train_ds(train_files, params['batch_size'])
valid_ds = get_train_ds(valid_files, params['batch_size'])
test_ds = get_test_ds(test_files, params['batch_size'])

In [ ]:
train_size, valid_size = get_ds_size(train_files), get_ds_size(valid_files)
test_size = get_ds_size(test_files)
print('the dataset consists of: {} training images, {} validation images, and {} test images'.
     format(train_size, valid_size, test_size))

In [ ]:
epoch_steps = train_size / params['batch_size'] 
valid_steps = valid_size / params['batch_size']
test_steps = 1.0 * test_size / params['batch_size'] 

In [ ]:
#calculate class weights

targets = train_df.groupby('target').count()['diagnosis'].to_list()
target_0 = targets[0]
target_1 = targets[1]
total = target_0 + target_1

class_weight_0 = (1 / target_0) * (total) / 2.0
class_weight_1 = (1 / target_1) * (total) / 2.0

class_weights = {0: class_weight_0, 1: class_weight_1}

initial_bias = np.log([target_1 / target_0])



In [ ]:
with strategy.scope():
    model = create_model(bias_output=initial_bias)
    model.summary()

# Training

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', #val_auc
                                patience=40,
                                mode='max',
                                restore_best_weights=True)


history = model.fit(
    train_ds,
#     batch_size=params['batch_size'],
    epochs= params['epochs'], 
    steps_per_epoch=epoch_steps,
    validation_data=valid_ds,
    validation_steps=valid_steps,
    class_weight=class_weights,
    callbacks=[early_stopping]
)


In [ ]:
predictions = model.predict(test_ds.map(lambda img, igs: img), steps=test_steps)  

In [ ]:
prediction_ids = next(iter(test_ds.
                          map(lambda img, ids:ids).
                          unbatch().
                          batch(test_size))).numpy().astype('str')

In [ ]:
prediction_dict = {
    'image_name': prediction_ids,
    'target': np.concatenate(predictions)
}
submission_ds = pd.DataFrame(prediction_dict)

In [ ]:
submission_ds.head()

In [ ]:
submission_ds.to_csv('submission.csv', index=False)

# Metrics

In [ ]:
def plot_metric(metric1, metric2, ylabel):
    plt.plot(history.history[metric1], label=metric1)
    plt.plot(history.history[metric2], label=metric2)
    plt.ylabel(ylabel)
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

In [ ]:
plot_metric('auc', 'val_auc', 'auc')
plot_metric('loss', 'val_loss', 'loss')
plot_metric('accuracy', 'val_accuracy', 'accuracy')

In [ ]:
history.history